In [19]:
import pandas as pd
from shapely.geometry import Point
from pyproj import CRS
import sys
sys.path.append(r"C:\Users\JoaoPereira\Desktop\pygeo\Geospatial-Ops")
import aux_functions as af
import importlib
importlib.reload(af)
from shapely.geometry import LineString
from shapely.ops import unary_union
from shapely.geometry import box
import os
import glob
import json
import ee
ee.Authenticate()
ee.Initialize()
import rasterio
from rasterio.features import shapes
from osgeo import gdal
import pickle
import os.path
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload 
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import geopandas as gpd
import os
import osmnx as ox
from shapely import wkt
import fiona
import requests

In [ ]:
#------------------------------------------------------
# Automate Landslide Susceptibility Mapping through the use of auxiliar functions from aux_functions.py
# In total there are 4 notebooks for this automation, this one has the purpose to prepare the data, including creating spatial joins between slope and in situ data,
# resample, clipping to the extended bounds, set crs, stack rasters (median,mean or sum) and perform IDW (when needed)."

# These notebooks ONLY uses free global available data

#@author: Joao Pereira  
#
#------------------------------------------------------

In [ ]:
#Global Landslide Inventory
#https://maps.nccs.nasa.gov/arcgis/apps/MapAndAppGallery/index.html?appid=574f26408683485799d02e857e5d9521

## Area of Interest

In [2]:
# AOI

# AUTOMATED ROAD NETWORK DOWNLOAD

# Define your custom polygon using WKT (Well-Known Text)

# use https://geojson.io/ to get the WKT from the chosen landslide inventory

custom_polygon = "POLYGON ((-122.62207922247859 38.01246655020722, -122.62207922247859 37.9967465230248, -122.60024982344561 37.9967465230248, -122.60024982344561 38.01246655020722, -122.62207922247859 38.01246655020722))"

# Convert the WKT polygon to a Shapely geometry
polygon_geometry = wkt.loads(custom_polygon)

In [6]:
# Use the polygon_geometry to extract the network
road = ox.graph_from_polygon(polygon_geometry, network_type="drive")

# Project the graph
prj_road = ox.project_graph(road)

# Save the network vector as a geopackage
ox.save_graph_geopackage(prj_road, filepath = r"E:\Spotlite_JPereira\Pessoal\LSM\AOI\SF_roadnetwork.gpkg")

In [13]:
# Replace this with the path to your GeoPackage
gpkg_path = r"E:\Spotlite_JPereira\Pessoal\LSM\AOI\SF_roadnetwork.gpkg"

layers = fiona.listlayers(gpkg_path)
print(layers)

['nodes', 'edges']


In [14]:
# Read the 'edges' layer from the GeoPackage
edges_gdf = gpd.read_file(gpkg_path, layer='edges')

geojson_path = r"E:\Spotlite_JPereira\Pessoal\LSM\AOI\SF_road_line_AOI.geojson"

edges_gdf.to_file(geojson_path, driver='GeoJSON')

In [16]:
# Create Road Pavement Buffer

line_gdf = gpd.read_file(geojson_path)

# Create a X-meter buffer with a flat cap style
buffer_distance = 35
buffered_gdf = line_gdf.copy()
buffered_gdf["geometry"] = line_gdf["geometry"].buffer(buffer_distance, cap_style=1)

# Dissolve all individual buffers into a single geometry
dissolved_buffer = unary_union(buffered_gdf["geometry"])

dissolved_buffer_gdf = gpd.GeoDataFrame(geometry=[dissolved_buffer], crs=line_gdf.crs)

dissolved_buffer_geojson_path = r"E:\Spotlite_JPereira\Pessoal\LSM\AOI\SF_road_line_AOI_buffer.geojson"
dissolved_buffer_gdf.to_file(dissolved_buffer_geojson_path, driver="GeoJSON")

In [18]:
gdf = dissolved_buffer_gdf.to_crs(4326)
gdf.to_file(r"E:\Spotlite_JPereira\Pessoal\LSM\AOI\SF_road_line_AOI_buffer_wgs84.geojson", driver="GeoJSON")

## Lithology

In [30]:
# Adapt WFS from https://mrdata.usgs.gov/geology/state/


# Define the WFS request URL
url = 'https://mrdata.usgs.gov/services/wfs/sgmc2?service=WFS&version=1.1.0&request=GetFeature&typeName=Lithology&outputFormat=text/xml; subtype=gml/3.1.1'
# Send the request
response = requests.get(url)
# Save the downloaded data to a file
with open(r"E:\Spotlite_JPereira\Pessoal\LSM\Lithology\US_Lithology_Mapping.gml", 'w') as file:
    file.write(response.text)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))